In [70]:
%pip install transformers
%pip install pandas
%pip install torch
%pip install torchvision

import pandas as pd
import os
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction
import numpy as np
import re
from torch.nn.functional import softmax
import torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [71]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
#print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
#print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 1.13.0
Using device: mps


Model 1: https://huggingface.co/roberta-base
Model 2: https://huggingface.co/vinai/bertweet-base

In [72]:
# class IntersentenceEvaluator():
#     def __init__(self, data, choices, model):
#         self.data = data
#         self.choices = choices
#         self.model = model
#         self.process_sentences()
#         self.tokenizer = AutoTokenizer.from_pretrained(model)
#         self.sent_encodings, self.word_encodings, self.mask_idxs = self.make_encodings() #store the encodings
#         self.model = AutoModelForMaskedLM.from_pretrained(model)

In [73]:
choices = {'bias':1, 'unbiased':2,'unrelated':3} 
file_path = 'male_refugee_inter.csv' #"drive/MyDrive/Final_templates.csv"#"drive/MyDrive/New_templates.csv"
model_name = 'distilroberta-base'
templates = pd.read_csv(file_path, sep=";")
#evaluator = IntersentenceEvaluator(templates.copy(), choices, model_name)
#evaluator.run_model_and_evaluate()

In [74]:
model_name = 'distilroberta-base'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
first_sent = "Modern pizza was invented in Naples, and the dish and its variants have since become popular in many countries."
next_sent = "It has become one of the most popular foods in the world"

encoding = tokenizer(first_sent, next_sent, return_tensors='pt', max_length=512, truncation = True, padding = 'max_length')

loss, logits = model(**encoding, next_sentence_label=torch.LongTensor([1]))

print(logits)

/Users/mae/opt/miniconda3/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py:1457: FutureWarning: The `next_sentence_label` argument is deprecated and will be removed in a future version, use `labels` instead.
  warnings.warn(


logits


In [76]:
#convert logit to probabilities 
probs = softmax(logits, dim=1)
probs

AttributeError: 'str' object has no attribute 'softmax'